<a href="https://colab.research.google.com/github/anon12a/ed-vae/blob/main/Main_Experiment_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import pandas as pd

from keras.layers import *
from keras.layers import Layer, Dense, Activation, Flatten, Lambda, Conv2D, Conv2DTranspose, BatchNormalization, Reshape
from keras.models import Model, Input, Sequential
from keras.callbacks import EarlyStopping
from keras.losses import mse
from keras import backend as K
from keras import optimizers, initializers, regularizers, constraints
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, KernelPCA
from keras.datasets import mnist, fashion_mnist

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
%matplotlib inline 

#Utilities

In [ ]:
def flatten_data(data):

  data = np.reshape(data,(-1, original_dim))
  
  return data

#Losses and Metrics

In [ ]:
def reconstruction_error(model):

    recon_error = mse(K.reshape(model.input, [-1]), K.reshape(model.output, [-1]))  
    recon_error *= original_dim
    recon_error = K.mean(recon_error)

    return recon_error     

In [ ]:
def kl_divergence(encoder):

    kl = K.mean(- 0.5 * K.sum(1 + encoder.get_output_at(1)[1] - K.square(encoder.get_output_at(1)[0]) - K.exp(encoder.get_output_at(1)[1]), axis=-1))

    return kl

In [ ]:
def per_example_reconstruction_error(model, data):

    decoded = model.predict(data, batch_size=batch_size)
    recon_error = mse(np.reshape(data,(-1,original_dim)), np.reshape(decoded,(-1,original_dim)))
    recon_error *= original_dim

    return tf.Session().run(recon_error)

In [ ]:
# reconstruction error of the expected latent representation 
def per_example_elr_reconstruction_error(encoder, decoder, data):

    z_mean, z_log_var, z = encoder.predict(data, batch_size=batch_size)
    decoded = decoder.predict(z_mean, batch_size=batch_size)
    recon_error = mse(np.reshape(data,(-1,original_dim)), np.reshape(decoded,(-1,original_dim)))
    recon_error *= original_dim

    return tf.Session().run(recon_error)

In [ ]:
def per_example_kl_divergence(encoder, data):

    z_mean, z_log_var, z = encoder.predict(data, batch_size=batch_size)

    kl = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl = K.sum(kl, axis=-1)
    kl *= -0.5

    return tf.Session().run(kl)

In [ ]:
def pr_auc(x, y):
    precision, recall, _ = precision_recall_curve(x, y, pos_label=1)
    pr_auc = auc(recall, precision)
    
    return pr_auc    

In [ ]:
def roc_auc(x, y):
    fpr, tpr, thresholds = roc_curve(x, y, pos_label=1)
    roc_auc = auc(fpr, tpr)

    return roc_auc 

# Networks

## Convolutional Autoencoder

In [ ]:
def conv_encoder(input_side=32, n_channels=3, representation_dim=512, filter_size = 64, representation_activation='sigmoid', intermediate_activation='relu', batch_norm = False):
  
    input_shape = (input_side, input_side, n_channels)

    input = Input(shape=input_shape)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(input)
    if batch_norm == True:
      x = BatchNormalization()(x)
    x = Activation(intermediate_activation)(x)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x) 
    x = Activation(intermediate_activation)(x)

    x = Flatten()(x)

    x = Dense(filter_size * input_side//4 * input_side//4)(x)
    if batch_norm == True:
      x = BatchNormalization()(x) 
    x = Activation(intermediate_activation)(x)

    x = Dense(representation_dim)(x)
    if batch_norm == True:
      x = BatchNormalization()(x)
    encoded = Activation(representation_activation)(x)   

    return Model(input, encoded, name="encoder")

In [ ]:
def conv_decoder(output_side=32, n_channels=3, representation_dim=512, filter_size = 64, activation='relu', batch_norm = False):

    encoded_input = Input(shape=(representation_dim,))

    x = Dense(filter_size * output_side//4 * output_side//4)(encoded_input)
    if batch_norm == True:
      x = BatchNormalization()(x) 
    x = Activation(activation)(x)

    conv_shape = (output_side//4, output_side//4, filter_size)
    x = Reshape(conv_shape)(x)
    
    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x)    
    x = Activation(activation)(x)

    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x)     
    x = Activation(activation)(x)

    x = Conv2DTranspose(n_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    if batch_norm == True:
      x = BatchNormalization()(x)    
    decoded = Activation('sigmoid')(x)

    return Model(encoded_input, decoded, name="decoder")

In [ ]:
def convolutional_autoencoder(encoder, decoder):

    input = Input(batch_shape=encoder.input_shape)
    output = decoder(encoder(input))

    cae = Model(input, output, name='cae')

    return cae

## Convolutional Variational Autoencoder

In [ ]:
def cvae_encoder(input_side=32, n_channels=3, representation_dim=512, filter_size = 64, intermediate_activation='relu', batch_norm=False):  
    input_shape = (input_side, input_side, n_channels)

    input = Input(shape=input_shape)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(input)
    x = Activation(intermediate_activation)(x)

    x = Conv2D(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Activation(intermediate_activation)(x)

    x = Flatten()(x)

    x = Dense(filter_size * input_side//4 * input_side//4)(x)
    x = Activation(intermediate_activation)(x)

    z_mean = Dense(representation_dim, name='z_mean')(x)
    z_log_var = Dense(representation_dim, name='z_log_var')(x)
    z = Lambda(sampling, output_shape=(representation_dim,), name='z')([z_mean, z_log_var])

    return Model(input, [z_mean, z_log_var, z], name="encoder")

In [ ]:
def cvae_decoder(output_side=32, n_channels=3, representation_dim=512, filter_size = 64, activation='relu', batch_norm=False):
    
    encoded_input = Input(shape=(representation_dim,))

    x = Dense(filter_size * output_side//4 * output_side//4)(encoded_input)
    x = Activation(activation)(x)

    conv_shape = (output_side//4, output_side//4, filter_size)
    x = Reshape(conv_shape)(x)

    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Activation(activation)(x)

    x = Conv2DTranspose(filter_size, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Activation(activation)(x)

    x = Conv2DTranspose(n_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    decoded = Activation('sigmoid')(x)

    return Model(encoded_input, decoded, name="decoder")

In [ ]:
def convolutional_variational_autoencoder(encoder, decoder):

    input = Input(batch_shape=encoder.input_shape)
    output = decoder(encoder(input)[2])

    cvae = Model(input, output, name='cvae')

    return cvae

In [ ]:
# Reparamaterization Trick
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

#Comparable Models

## Convolutional Autoencoder

In [ ]:
def cae_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, input_side, n_channels, latent_dim, filter_size, batch_norm = False):

  encoder = conv_encoder(input_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  decoder = conv_decoder(output_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  cae = convolutional_autoencoder(encoder, decoder) 

  #encoder.summary()
  #decoder.summary() 

  recon_error = reconstruction_error(cae)

  cae.add_loss(recon_error)
  adam = optimizers.Adam(lr=learning_rate)
  cae.compile(optimizer=adam)

  es = EarlyStopping(monitor='val_loss', mode='min', patience = 20, verbose=1)

  history = cae.fit(x_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(x_val, None),
                  callbacks=[es],
                  verbose=0)
  
  rec_results = per_example_reconstruction_error(cae, x_test)

  pr_auc_rec_result = pr_auc(y_test, rec_results)
  roc_auc_rec_result = roc_auc(y_test, rec_results)

  return pr_auc_rec_result, roc_auc_rec_result, pr_auc_rec_result, roc_auc_rec_result, pr_auc_rec_result, roc_auc_rec_result

## PCA (via SVD)

In [ ]:
def pca_1(x_train, y_train, x_val, x_test, y_test, input_side, n_channels, latent_dim):

    pca = PCA(n_components=latent_dim, svd_solver='full')

    pca.fit(flatten_data(x_train))
    encoded = pca.transform(flatten_data(x_test))
    recon = pca.inverse_transform(encoded)

    recon_error = mse(flatten_data(x_test), recon)
    recon_error *= original_dim
    recon_error = tf.Session().run(recon_error)

    pr_auc_result = pr_auc(y_test, recon_error)
    roc_auc_result = roc_auc(y_test, recon_error)

    return pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result

## KernalPCA

In [ ]:
def kpca_1(x_train, y_train, x_val, x_test, y_test, input_side, n_channels, latent_dim):

    kpca = KernelPCA(n_components=latent_dim, kernel='rbf', fit_inverse_transform="True")

    kpca.fit(flatten_data(x_train))
    encoded = kpca.transform(flatten_data(x_test))
    recon = kpca.inverse_transform(encoded)

    recon_error = mse(flatten_data(x_test), recon)
    recon_error *= original_dim
    recon_error = tf.Session().run(recon_error)

    pr_auc_result = pr_auc(y_test, recon_error)
    roc_auc_result = roc_auc(y_test, recon_error)

    return pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result

## Deep Structured Energy Based Anomaly Detection (DSEBM)

In [ ]:
# From the following implementation: https://github.com/izikgo/AnomalyDetectionTransformations/blob/master/models/dsebm.py
# Licensed under the MIT License
class Prior(Layer):
    def __init__(self,
                 bias_initializer='zeros',
                 bias_regularizer=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(Prior, self).__init__(**kwargs)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.bias = None

    def build(self, input_shape):
        self.bias = self.add_weight(shape=input_shape[1:],
                                    initializer=self.bias_initializer,
                                    name='bias',
                                    regularizer=self.bias_regularizer,
                                    constraint=self.bias_constraint)
        super(Prior, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        return input_shape[0], 1

    def call(self, x, **kwargs):
        return K.sum(K.batch_flatten(K.square(K.bias_add(x, -self.bias))), axis=-1, keepdims=True)


def create_energy_model(encoder_mdl):
    x_in = Input(batch_shape=encoder_mdl.input_shape)

    encoded = encoder_mdl(x_in)
    prior = Prior()(x_in)
    energy = Lambda(lambda args: 0.5*args[0] - K.sum(K.batch_flatten(args[1]), axis=-1, keepdims=True),
                    output_shape=(1,))([prior, encoded])
    return Model(x_in, energy)


def create_reconstruction_model(energy_mdl):
    x_in = Input(batch_shape=energy_mdl.input_shape)
    x = GaussianNoise(stddev=.5)(x_in)  # only active in training
    energy = energy_mdl(x)
    rec = Lambda(lambda args: args[1] - K.gradients(args[0], args[1]),
                 output_shape=energy_mdl.input_shape[1:])([energy, x])

    return Model(x_in, rec)

In [ ]:
def dsebm_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, input_side, n_channels, latent_dim, filter_size):
  
  encoder_mdl = conv_encoder(input_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size=filter_size)
  energy_mdl = create_energy_model(encoder_mdl)
  reconstruction_mdl = create_reconstruction_model(energy_mdl)

  adam = optimizers.Adam(lr=learning_rate)
  reconstruction_mdl.compile(optimizer=adam, loss="mse")

  es = EarlyStopping(monitor='val_loss', mode='min', patience = 20, verbose=1)

  history = reconstruction_mdl.fit(x=x_train, y=x_train, 
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(x_val, x_val),
                  callbacks=[es],
                  verbose=0)
  
  scores = -energy_mdl.predict(x_test, batch_size)

  pr_auc_result = pr_auc(y_test, -scores)
  roc_auc_result = roc_auc(y_test, -scores)

  return pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result, pr_auc_result, roc_auc_result

# ED-VAE and VAE

In [ ]:
def edvae_vae_comparison(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, input_side, n_channels, latent_dim, filter_size, batch_norm=False, beta = 1.0):

  encoder = cvae_encoder(input_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  decoder = cvae_decoder(output_side=image_shape[0], n_channels=image_shape[2], representation_dim=latent_dim, filter_size = filter_size, batch_norm = batch_norm)
  cae = convolutional_variational_autoencoder(encoder, decoder) 

  #encoder.summary()
  #decoder.summary() 

  recon_error = reconstruction_error(cae)
  kl = kl_divergence(encoder)
  loss = beta * kl + recon_error

  cae.add_loss(loss)
  
  adam = optimizers.Adam(lr=learning_rate)
  cae.compile(optimizer=adam)

  es = EarlyStopping(monitor='val_loss', mode='min', patience = 20, verbose=1)

  history = cae.fit(x_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(x_val, None),
                  callbacks=[es],
                  verbose=0)
  
  # vae results 
  kl_results = per_example_kl_divergence(encoder, x_test)
  rec_results = per_example_reconstruction_error(cae, x_test)
  elbo_results = beta * kl_results + rec_results

  pr_auc_rec_result = pr_auc(y_test, rec_results)
  roc_auc_rec_result = roc_auc(y_test, rec_results)

  pr_auc_elbo_result = pr_auc(y_test, elbo_results)
  roc_auc_elbo_result = roc_auc(y_test, elbo_results)

  pr_auc_kl_result = pr_auc(y_test, kl_results)
  roc_auc_kl_result = roc_auc(y_test, kl_results) 
    
  # ed-vae results
  kl_results = per_example_kl_divergence(encoder, x_test)
  rec_results = per_example_elr_reconstruction_error(encoder, decoder, x_test)
  elbo_results = beta * kl_results + rec_results

  elr_pr_auc_rec_result = pr_auc(y_test, rec_results)
  elr_roc_auc_rec_result = roc_auc(y_test, rec_results)

  elr_pr_auc_elbo_result = pr_auc(y_test, elbo_results)
  elr_roc_auc_elbo_result = roc_auc(y_test, elbo_results)

  elr_pr_auc_kl_result = pr_auc(y_test, kl_results )
  elr_roc_auc_kl_result = roc_auc(y_test, kl_results )

  return np.array([[pr_auc_rec_result, roc_auc_rec_result, pr_auc_elbo_result, roc_auc_elbo_result, pr_auc_kl_result, roc_auc_kl_result],
                   [elr_pr_auc_rec_result, elr_roc_auc_rec_result, elr_pr_auc_elbo_result, elr_roc_auc_elbo_result, elr_pr_auc_kl_result, elr_roc_auc_kl_result]])

# Data Preperation

In [ ]:
def load_mnist():

  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
  x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

  #print('x_train shape:', x_train.shape)
  #print(x_train.shape[0], 'train samples')
  #print(x_test.shape[0], 'test samples')

  return x_train, y_train, x_test, y_test

In [ ]:
def load_fashion_mnist():

  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  x_train =  x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
  x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

  #print('x_train shape:', x_train.shape)
  #print(x_train.shape[0], 'train samples')
  #print(x_test.shape[0], 'test samples')

  return x_train, y_train, x_test, y_test

In [ ]:
# create training set without anomalies and test set with a ratio p of anomalies. E.g. p = 0.2 
# creates a test set where 20% of the test set are anomalies
def create_data(X_train, Y_train, X_test, Y_test, normal_class, p = .1):

    X_norm_train_filter = np.where(np.isin(Y_train, normal_class))
    X_norm_test_filter = np.where(np.isin(Y_test, normal_class))
    X_anom_test_filter = np.where(np.isin(Y_test, normal_class, invert = True))

    # a = (# of normal instances)/(1 - ratio of outliers)) * ratio of outliers
    # e.g. if p = .2 20% of the test set is anomalies
    a = math.floor(((len(X_norm_test_filter[0]))/(1-p))*p)

    X_anom_test_filter = (np.random.choice(X_anom_test_filter[0], a, replace=False),)
    X_test_filter = (np.concatenate((X_norm_test_filter[0], X_anom_test_filter[0]), axis=None),)

    X_train, Y_train = X_train[X_norm_train_filter[0]], Y_train[X_norm_train_filter]
    X_test_norm, Y_test_norm = X_test[X_norm_test_filter[0]], Y_test[X_norm_test_filter]
    Y_test_class_labels = Y_test[X_test_filter]
    X_test, Y_test, = X_test[X_test_filter[0]], Y_test[X_test_filter]

    Y_test[np.isin(Y_test_class_labels, normal_class)] = 0
    Y_test[np.isin(Y_test_class_labels, normal_class, invert = True)] = 1
    
    return X_train, Y_train, X_test, Y_test, Y_test_class_labels

# Configuration

In [ ]:
image_shape = (28, 28, 1)
original_dim = image_shape[0] * image_shape[1] * image_shape[2]
input_shape = image_shape
batch_size = 128
latent_dim = 32
beta = 1.0
epochs = 1000
learning_rate = 0.0001
activation = 'relu'
filter_size = 32
p = 0.2

# Experiments

In [ ]:
normal_classes = np.arange(0,10)
num_models = 6
num_measures = 6
reps = 10

In [ ]:
def models(x_train, y_train, x_val, x_test, y_test):

  pca_1_results = pca_1(x_train, y_train, x_val, x_test, y_test, image_shape[0], image_shape[2], latent_dim)
  kpca_1_results = kpca_1(x_train, y_train, x_val, x_test, y_test, image_shape[0], image_shape[2], latent_dim)
  dsebm_1_results = dsebm_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, image_shape[0], image_shape[2], latent_dim, filter_size)
  cae_1_results = cae_1(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, image_shape[0], image_shape[2], latent_dim, filter_size, batch_norm=False)
  edvae_vae_comparison_results = edvae_vae_comparison(x_train, y_train, x_val, x_test, y_test, epochs, batch_size, learning_rate, image_shape[0], image_shape[2], latent_dim, filter_size, beta = beta)
  
  return pca_1_results, kpca_1_results, dsebm_1_results, cae_1_results, edvae_vae_comparison_results[0], edvae_vae_comparison_results[1]

In [ ]:
results = np.zeros((reps, len(normal_classes), num_models, num_measures))

from tqdm import tqdm

for rep in tqdm(range(reps), desc="reps"):
  for normal in tqdm(range(len(normal_classes)), desc="anomaly"):
    x_train, y_train, x_test, y_test = load_fashion_mnist()
    x_train, y_train, x_test, y_test, y_test_class_labels = create_data(x_train, y_train, x_test, y_test, normal_classes[normal], p = p)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    results[rep, normal] = models(x_train, y_train, x_val, x_test, y_test)
    tf.keras.backend.clear_session() #release gpu memory

anomaly:   0%|          | 0/10 [00:00<?, ?it/s]

4423680/4422102 [==============================] - 0s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 00205: early stopping


/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


Epoch 00232: early stopping


/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


...


anomaly:  90%|█████████ | 9/10 [41:54<04:43, 283.32s/it]/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:530: RuntimeWarning: invalid value encountered in multiply
  v *= signs[:, np.newaxis]


Epoch 00227: early stopping


/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))
/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


Epoch 00321: early stopping



reps: 100%|██████████| 10/10 [6:41:20<00:00, 2408.02s/it]


# Results - ED-VAE vs. Comparable Models

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
index_labels = normal_classes

In [ ]:
avg_results = np.mean(results, axis=0)
std_results = np.std(results, axis=0)
avg_class_results = np.mean(results, axis=(0, 1))
std_class_results = np.std(results, axis=(0, 1))

In [ ]:
pr_header = [np.array(['pca','pca',
                    'kpca','kpca',
                    'dsebm','dsebm',
                    'ae','ae',
                    'vae', 'vae',             
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          ])] 

In [ ]:
auc_header = [np.array(['pca','pca',
                    'kpca','kpca',
                    'dsebm','dsebm',
                    'ae','ae',
                    'vae', 'vae',             
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          'auc_mean','auc_std',
          ])] 

## $ELBO$ 
Compare ELBO of VAE and ED-VAE to results of competitive models


In [ ]:
# AUC
avg_auc_elbo_class_results = pd.DataFrame((avg_class_results[0,3], std_class_results[0,3], 
                                  avg_class_results[1,3], std_class_results[1,3],
                                  avg_class_results[2,3], std_class_results[2,3],  
                                  avg_class_results[3,3], std_class_results[3,3], 
                                  avg_class_results[4,3], std_class_results[4,3], 
                                  avg_class_results[5,3], std_class_results[5,3], 
                                  ), 
                                  index = auc_header).T
avg_auc_elbo_class_results = avg_auc_elbo_class_results.rename(index={0: "mean"})
avg_auc_elbo_final_results = pd.DataFrame((avg_results[:,0,3], std_results[:,0,3],
                                  avg_results[:,1,3], std_results[:,1,3], 
                                  avg_results[:,2,3], std_results[:,2,3], 
                                  avg_results[:,3,3], std_results[:,3,3], 
                                  avg_results[:,4,3], std_results[:,4,3],  
                                  avg_results[:,5,3], std_results[:,5,3], 
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_elbo_final_results.append(avg_auc_elbo_class_results)

pca                kpca               dsebm                  ae  \
      auc_mean   auc_std  auc_mean   auc_std  auc_mean   auc_std  auc_mean   
0     0.898339  0.005398  0.906345  0.006592  0.872652  0.019989  0.896966   
1     0.985436  0.001879  0.982097  0.002359  0.784110  0.134305  0.979720   
2     0.887165  0.008119  0.890545  0.007060  0.833402  0.013873  0.866617   
3     0.919152  0.007362  0.925635  0.011317  0.913408  0.015568  0.905385   
4     0.885009  0.013492  0.884886  0.014163  0.875427  0.019237  0.887289   
5     0.886346  0.009993  0.888076  0.008158  0.872494  0.008884  0.876394   
6     0.813194  0.012386  0.821431  0.009973  0.756203  0.019512  0.776370   
7     0.984449  0.002424  0.983995  0.002469  0.953105  0.057234  0.980500   
8     0.836911  0.006764  0.839565  0.008515  0.795964  0.035885  0.821306   
9     0.970721  0.007420  0.966373  0.006154  0.972514  0.006888  0.971163   
mean  0.906672  0.056695  0.908895  0.053946  0.862928  0.083847  0.896171   

                     vae              ed-vae            
       auc_std  auc_mean   auc_std  auc_mean   auc_std  
0     0.012320  0.906740  0.007390  0.911173  0.007674  
1     0.006758  0.985576  0.002004  0.987114  0.001906  
2     0.013023  0.884042  0.007990  0.891146  0.007604  
3     0.015106  0.924055  0.009302  0.927871  0.008745  
4     0.009809  0.899538  0.013268  0.908718  0.013040  
5     0.006077  0.893064  0.008791  0.895072  0.007228  
6     0.017384  0.823253  0.012089  0.830588  0.011563  
7     0.002904  0.982244  0.003357  0.985006  0.002306  
8     0.023230  0.835084  0.011485  0.848064  0.012047  
9     0.007767  0.963205  0.007989  0.970858  0.006165  
mean  0.065376  0.909680  0.053844  0.915561  0.051721

In [ ]:
# PR-AUC
avg_pr_elbo_class_results = pd.DataFrame((avg_class_results[0,2], std_class_results[0,2], 
                                  avg_class_results[1,2], std_class_results[1,2],
                                  avg_class_results[2,2], std_class_results[2,2],  
                                  avg_class_results[3,2], std_class_results[3,2], 
                                  avg_class_results[4,2], std_class_results[4,2], 
                                  avg_class_results[5,2], std_class_results[5,2], 
                                  ), 
                                  index = pr_header).T
avg_pr_elbo_class_results = avg_pr_elbo_class_results.rename(index={0: "mean"})
avg_pr_elbo_final_results = pd.DataFrame((avg_results[:,0,2], std_results[:,0,2],
                                  avg_results[:,1,2], std_results[:,1,2], 
                                  avg_results[:,2,2], std_results[:,2,2], 
                                  avg_results[:,3,2], std_results[:,3,2], 
                                  avg_results[:,4,2], std_results[:,4,2],  
                                  avg_results[:,5,2], std_results[:,5,2], 
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_elbo_final_results.append(avg_pr_elbo_class_results)

pca                   kpca                  dsebm             \
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std   
0       0.698673   0.013300    0.730707   0.014845    0.674400   0.038337   
1       0.940467   0.006721    0.933861   0.007438    0.638965   0.167223   
2       0.708055   0.012931    0.751151   0.009973    0.624861   0.022804   
3       0.792979   0.018098    0.828515   0.020055    0.816480   0.021882   
4       0.762130   0.016841    0.763155   0.017801    0.711659   0.050418   
5       0.825434   0.009205    0.811392   0.007588    0.766110   0.016099   
6       0.599379   0.020939    0.624451   0.021465    0.470427   0.046672   
7       0.963140   0.004338    0.960817   0.004519    0.911702   0.084214   
8       0.614004   0.013122    0.658172   0.013313    0.464933   0.056769   
9       0.931633   0.011673    0.913366   0.012572    0.910897   0.019187   
mean    0.783589   0.125907    0.797559   0.109070    0.699043   0.164755   

              ae                    vae                 ed-vae             
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std  
0       0.711722   0.015720    0.722367   0.016622    0.721989   0.018962  
1       0.919310   0.021887    0.944248   0.007205    0.947529   0.007014  
2       0.694003   0.030495    0.741947   0.010923    0.749466   0.012310  
3       0.802410   0.025692    0.824475   0.018503    0.828265   0.018536  
4       0.740442   0.015772    0.786449   0.018267    0.799682   0.017247  
5       0.763120   0.008826    0.816291   0.007996    0.811389   0.007388  
6       0.508540   0.022887    0.639569   0.023570    0.645248   0.022122  
7       0.947319   0.003903    0.958223   0.006442    0.962574   0.004556  
8       0.526791   0.041653    0.592811   0.020648    0.608472   0.021543  
9       0.917540   0.028600    0.910692   0.014042    0.919511   0.011703  
mean    0.753120   0.147324    0.793707   0.117990    0.799412   0.116207

## $R_{error}$ 
Compare Reconstruction Error of VAE and ED-VAE to results of competitive models

In [ ]:
# AUC
avg_auc_rec_class_results = pd.DataFrame((avg_class_results[0,1], std_class_results[0,1], 
                                  avg_class_results[1,1], std_class_results[1,1],
                                  avg_class_results[2,1], std_class_results[2,1],  
                                  avg_class_results[3,1], std_class_results[3,1], 
                                  avg_class_results[4,1], std_class_results[4,1], 
                                  avg_class_results[5,1], std_class_results[5,1], 
                                  ), 
                                  index = auc_header).T
avg_auc_rec_class_results = avg_auc_rec_class_results.rename(index={0: "mean"})
avg_auc_rec_final_results = pd.DataFrame((avg_results[:,0,1], std_results[:,0,1], 
                                  avg_results[:,1,1], std_results[:,1,1],
                                  avg_results[:,2,1], std_results[:,2,1],  
                                  avg_results[:,3,1], std_results[:,3,1], 
                                  avg_results[:,4,1], std_results[:,4,1], 
                                  avg_results[:,5,1], std_results[:,5,1], 
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_rec_final_results.append(avg_auc_rec_class_results)

pca                kpca               dsebm                  ae  \
      auc_mean   auc_std  auc_mean   auc_std  auc_mean   auc_std  auc_mean   
0     0.898339  0.005398  0.906345  0.006592  0.872652  0.019989  0.896966   
1     0.985436  0.001879  0.982097  0.002359  0.784110  0.134305  0.979720   
2     0.887165  0.008119  0.890545  0.007060  0.833402  0.013873  0.866617   
3     0.919152  0.007362  0.925635  0.011317  0.913408  0.015568  0.905385   
4     0.885009  0.013492  0.884886  0.014163  0.875427  0.019237  0.887289   
5     0.886346  0.009993  0.888076  0.008158  0.872494  0.008884  0.876394   
6     0.813194  0.012386  0.821431  0.009973  0.756203  0.019512  0.776370   
7     0.984449  0.002424  0.983995  0.002469  0.953105  0.057234  0.980500   
8     0.836911  0.006764  0.839565  0.008515  0.795964  0.035885  0.821306   
9     0.970721  0.007420  0.966373  0.006154  0.972514  0.006888  0.971163   
mean  0.906672  0.056695  0.908895  0.053946  0.862928  0.083847  0.896171   

                     vae              ed-vae            
       auc_std  auc_mean   auc_std  auc_mean   auc_std  
0     0.012320  0.896063  0.007094  0.901122  0.007483  
1     0.006758  0.982152  0.002555  0.985460  0.002292  
2     0.013023  0.878164  0.008535  0.886226  0.007555  
3     0.015106  0.915553  0.007754  0.920695  0.007345  
4     0.009809  0.901077  0.013091  0.911860  0.012613  
5     0.006077  0.884665  0.008748  0.885300  0.007125  
6     0.017384  0.819804  0.014603  0.826755  0.013618  
7     0.002904  0.977047  0.004266  0.981493  0.002973  
8     0.023230  0.812620  0.010624  0.827066  0.011497  
9     0.007767  0.959224  0.010769  0.969056  0.007599  
mean  0.065376  0.902637  0.056620  0.909503  0.054988

In [ ]:
# PR-AUC
avg_pr_rec_class_results = pd.DataFrame((avg_class_results[0,0], std_class_results[0,0], 
                                  avg_class_results[1,0], std_class_results[1,0],
                                  avg_class_results[2,0], std_class_results[2,0],  
                                  avg_class_results[3,0], std_class_results[3,0], 
                                  avg_class_results[4,0], std_class_results[4,0], 
                                  avg_class_results[5,0], std_class_results[5,0], 
                                  ), 
                                  index = pr_header).T
avg_pr_rec_class_results = avg_pr_rec_class_results.rename(index={0: "mean"})
avg_pr_rec_final_results = pd.DataFrame((avg_results[:,0,0], std_results[:,0,0], 
                                  avg_results[:,1,0], std_results[:,1,0],
                                  avg_results[:,2,0], std_results[:,2,0],  
                                  avg_results[:,3,0], std_results[:,3,0], 
                                  avg_results[:,4,0], std_results[:,4,0], 
                                  avg_results[:,5,0], std_results[:,5,0], 
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_rec_final_results.append(avg_pr_rec_class_results)

pca                   kpca                  dsebm             \
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std   
0       0.698673   0.013300    0.730707   0.014845    0.674400   0.038337   
1       0.940467   0.006721    0.933861   0.007438    0.638965   0.167223   
2       0.708055   0.012931    0.751151   0.009973    0.624861   0.022804   
3       0.792979   0.018098    0.828515   0.020055    0.816480   0.021882   
4       0.762130   0.016841    0.763155   0.017801    0.711659   0.050418   
5       0.825434   0.009205    0.811392   0.007588    0.766110   0.016099   
6       0.599379   0.020939    0.624451   0.021465    0.470427   0.046672   
7       0.963140   0.004338    0.960817   0.004519    0.911702   0.084214   
8       0.614004   0.013122    0.658172   0.013313    0.464933   0.056769   
9       0.931633   0.011673    0.913366   0.012572    0.910897   0.019187   
mean    0.783589   0.125907    0.797559   0.109070    0.699043   0.164755   

              ae                    vae                 ed-vae             
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std  
0       0.711722   0.015720    0.686710   0.017583    0.683710   0.019027  
1       0.919310   0.021887    0.936850   0.007662    0.943020   0.007366  
2       0.694003   0.030495    0.721591   0.013055    0.727865   0.014579  
3       0.802410   0.025692    0.801492   0.017408    0.805554   0.017027  
4       0.740442   0.015772    0.780542   0.019491    0.793460   0.018910  
5       0.763120   0.008826    0.809740   0.008844    0.802923   0.007537  
6       0.508540   0.022887    0.631358   0.022082    0.635030   0.021455  
7       0.947319   0.003903    0.950122   0.008087    0.956836   0.005809  
8       0.526791   0.041653    0.549982   0.020126    0.564549   0.021504  
9       0.917540   0.028600    0.904508   0.016123    0.914632   0.013445  
mean    0.753120   0.147324    0.777290   0.126473    0.782758   0.126155

## Formated for Publication
Mean and SEM

In [ ]:
# returns the list for df.style.apply() method
def highlight_max(s):
    s = s.str.split(pat='±',expand=True)[0]
    is_max = s == s.max()
    return ['font-weight: bold' if cell else '' for cell in is_max]

### $ELBO$ 
 Compare ELBO of VAE and ED-VAE to results of competitive models

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_elbo_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,89.8±0.2,90.6±0.2,87.3±0.6,89.7±0.4,90.7±0.2,91.1±0.2
1,98.5±0.1,98.2±0.1,78.4±4.2,98.0±0.2,98.6±0.1,98.7±0.1
2,88.7±0.3,89.1±0.2,83.3±0.4,86.7±0.4,88.4±0.3,89.1±0.2
3,91.9±0.2,92.6±0.4,91.3±0.5,90.5±0.5,92.4±0.3,92.8±0.3
4,88.5±0.4,88.5±0.4,87.5±0.6,88.7±0.3,90.0±0.4,90.9±0.4
5,88.6±0.3,88.8±0.3,87.2±0.3,87.6±0.2,89.3±0.3,89.5±0.2
6,81.3±0.4,82.1±0.3,75.6±0.6,77.6±0.5,82.3±0.4,83.1±0.4
7,98.4±0.1,98.4±0.1,95.3±1.8,98.1±0.1,98.2±0.1,98.5±0.1
8,83.7±0.2,84.0±0.3,79.6±1.1,82.1±0.7,83.5±0.4,84.8±0.4
9,97.1±0.2,96.6±0.2,97.3±0.2,97.1±0.2,96.3±0.3,97.1±0.2


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,89.8±0.2,90.6±0.2,87.3±0.6,89.7±0.4,90.7±0.2,91.1±0.2
1,98.5±0.1,98.2±0.1,78.4±4.2,98.0±0.2,98.6±0.1,98.7±0.1
2,88.7±0.3,89.1±0.2,83.3±0.4,86.7±0.4,88.4±0.3,89.1±0.2
3,91.9±0.2,92.6±0.4,91.3±0.5,90.5±0.5,92.4±0.3,92.8±0.3
4,88.5±0.4,88.5±0.4,87.5±0.6,88.7±0.3,90.0±0.4,90.9±0.4
5,88.6±0.3,88.8±0.3,87.2±0.3,87.6±0.2,89.3±0.3,89.5±0.2
6,81.3±0.4,82.1±0.3,75.6±0.6,77.6±0.5,82.3±0.4,83.1±0.4
7,98.4±0.1,98.4±0.1,95.3±1.8,98.1±0.1,98.2±0.1,98.5±0.1
8,83.7±0.2,84.0±0.3,79.6±1.1,82.1±0.7,83.5±0.4,84.8±0.4
9,97.1±0.2,96.6±0.2,97.3±0.2,97.1±0.2,96.3±0.3,97.1±0.2


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_elbo_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_final_results.keys().get_level_values(0)[::2]).T  
df = df.append(pd.DataFrame((
    #round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,69.9±0.4,73.1±0.5,67.4±1.2,71.2±0.5,72.2±0.5,72.2±0.6
1,94.0±0.2,93.4±0.2,63.9±5.3,91.9±0.7,94.4±0.2,94.8±0.2
2,70.8±0.4,75.1±0.3,62.5±0.7,69.4±1.0,74.2±0.3,74.9±0.4
3,79.3±0.6,82.9±0.6,81.6±0.7,80.2±0.8,82.4±0.6,82.8±0.6
4,76.2±0.5,76.3±0.6,71.2±1.6,74.0±0.5,78.6±0.6,80.0±0.5
5,82.5±0.3,81.1±0.2,76.6±0.5,76.3±0.3,81.6±0.3,81.1±0.2
6,59.9±0.7,62.4±0.7,47.0±1.5,50.9±0.7,64.0±0.7,64.5±0.7
7,96.3±0.1,96.1±0.1,91.2±2.7,94.7±0.1,95.8±0.2,96.3±0.1
8,61.4±0.4,65.8±0.4,46.5±1.8,52.7±1.3,59.3±0.7,60.8±0.7
9,93.2±0.4,91.3±0.4,91.1±0.6,91.8±0.9,91.1±0.4,92.0±0.4


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,69.9±0.4,73.1±0.5,67.4±1.2,71.2±0.5,72.2±0.5,72.2±0.6
1,94.0±0.2,93.4±0.2,63.9±5.3,91.9±0.7,94.4±0.2,94.8±0.2
2,70.8±0.4,75.1±0.3,62.5±0.7,69.4±1.0,74.2±0.3,74.9±0.4
3,79.3±0.6,82.9±0.6,81.6±0.7,80.2±0.8,82.4±0.6,82.8±0.6
4,76.2±0.5,76.3±0.6,71.2±1.6,74.0±0.5,78.6±0.6,80.0±0.5
5,82.5±0.3,81.1±0.2,76.6±0.5,76.3±0.3,81.6±0.3,81.1±0.2
6,59.9±0.7,62.4±0.7,47.0±1.5,50.9±0.7,64.0±0.7,64.5±0.7
7,96.3±0.1,96.1±0.1,91.2±2.7,94.7±0.1,95.8±0.2,96.3±0.1
8,61.4±0.4,65.8±0.4,46.5±1.8,52.7±1.3,59.3±0.7,60.8±0.7
9,93.2±0.4,91.3±0.4,91.1±0.6,91.8±0.9,91.1±0.4,92.0±0.4


### $R_{error}$ 
Compare Reconstruction Error of VAE and ED-VAE to results of competitive models

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_rec_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,89.8±0.2,90.6±0.2,87.3±0.6,89.7±0.4,89.6±0.2,90.1±0.2
1,98.5±0.1,98.2±0.1,78.4±4.2,98.0±0.2,98.2±0.1,98.5±0.1
2,88.7±0.3,89.1±0.2,83.3±0.4,86.7±0.4,87.8±0.3,88.6±0.2
3,91.9±0.2,92.6±0.4,91.3±0.5,90.5±0.5,91.6±0.2,92.1±0.2
4,88.5±0.4,88.5±0.4,87.5±0.6,88.7±0.3,90.1±0.4,91.2±0.4
5,88.6±0.3,88.8±0.3,87.2±0.3,87.6±0.2,88.5±0.3,88.5±0.2
6,81.3±0.4,82.1±0.3,75.6±0.6,77.6±0.5,82.0±0.5,82.7±0.4
7,98.4±0.1,98.4±0.1,95.3±1.8,98.1±0.1,97.7±0.1,98.1±0.1
8,83.7±0.2,84.0±0.3,79.6±1.1,82.1±0.7,81.3±0.3,82.7±0.4
9,97.1±0.2,96.6±0.2,97.3±0.2,97.1±0.2,95.9±0.3,96.9±0.2


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,89.8±0.2,90.6±0.2,87.3±0.6,89.7±0.4,89.6±0.2,90.1±0.2
1,98.5±0.1,98.2±0.1,78.4±4.2,98.0±0.2,98.2±0.1,98.5±0.1
2,88.7±0.3,89.1±0.2,83.3±0.4,86.7±0.4,87.8±0.3,88.6±0.2
3,91.9±0.2,92.6±0.4,91.3±0.5,90.5±0.5,91.6±0.2,92.1±0.2
4,88.5±0.4,88.5±0.4,87.5±0.6,88.7±0.3,90.1±0.4,91.2±0.4
5,88.6±0.3,88.8±0.3,87.2±0.3,87.6±0.2,88.5±0.3,88.5±0.2
6,81.3±0.4,82.1±0.3,75.6±0.6,77.6±0.5,82.0±0.5,82.7±0.4
7,98.4±0.1,98.4±0.1,95.3±1.8,98.1±0.1,97.7±0.1,98.1±0.1
8,83.7±0.2,84.0±0.3,79.6±1.1,82.1±0.7,81.3±0.3,82.7±0.4
9,97.1±0.2,96.6±0.2,97.3±0.2,97.1±0.2,95.9±0.3,96.9±0.2


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_rec_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,pca,kpca,dsebm,ae,vae,ed-vae
0,69.9±0.4,73.1±0.5,67.4±1.2,71.2±0.5,68.7±0.6,68.4±0.6
1,94.0±0.2,93.4±0.2,63.9±5.3,91.9±0.7,93.7±0.2,94.3±0.2
2,70.8±0.4,75.1±0.3,62.5±0.7,69.4±1.0,72.2±0.4,72.8±0.5
3,79.3±0.6,82.9±0.6,81.6±0.7,80.2±0.8,80.1±0.6,80.6±0.5
4,76.2±0.5,76.3±0.6,71.2±1.6,74.0±0.5,78.1±0.6,79.3±0.6
5,82.5±0.3,81.1±0.2,76.6±0.5,76.3±0.3,81.0±0.3,80.3±0.2
6,59.9±0.7,62.4±0.7,47.0±1.5,50.9±0.7,63.1±0.7,63.5±0.7
7,96.3±0.1,96.1±0.1,91.2±2.7,94.7±0.1,95.0±0.3,95.7±0.2
8,61.4±0.4,65.8±0.4,46.5±1.8,52.7±1.3,55.0±0.6,56.5±0.7
9,93.2±0.4,91.3±0.4,91.1±0.6,91.8±0.9,90.5±0.5,91.5±0.4


In [ ]:
df.style.apply(highlight_max, axis=1)

,pca,kpca,dsebm,ae,vae,ed-vae
0,69.9±0.4,73.1±0.5,67.4±1.2,71.2±0.5,68.7±0.6,68.4±0.6
1,94.0±0.2,93.4±0.2,63.9±5.3,91.9±0.7,93.7±0.2,94.3±0.2
2,70.8±0.4,75.1±0.3,62.5±0.7,69.4±1.0,72.2±0.4,72.8±0.5
3,79.3±0.6,82.9±0.6,81.6±0.7,80.2±0.8,80.1±0.6,80.6±0.5
4,76.2±0.5,76.3±0.6,71.2±1.6,74.0±0.5,78.1±0.6,79.3±0.6
5,82.5±0.3,81.1±0.2,76.6±0.5,76.3±0.3,81.0±0.3,80.3±0.2
6,59.9±0.7,62.4±0.7,47.0±1.5,50.9±0.7,63.1±0.7,63.5±0.7
7,96.3±0.1,96.1±0.1,91.2±2.7,94.7±0.1,95.0±0.3,95.7±0.2
8,61.4±0.4,65.8±0.4,46.5±1.8,52.7±1.3,55.0±0.6,56.5±0.7
9,93.2±0.4,91.3±0.4,91.1±0.6,91.8±0.9,90.5±0.5,91.5±0.4


# Results - ED-VAE vs. VAE

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
index_labels = normal_classes

In [ ]:
avg_results = np.mean(results, axis=0)
std_results = np.std(results, axis=0)
avg_class_results = np.mean(results, axis=(0, 1))
std_class_results = np.std(results, axis=(0, 1))

In [ ]:
avg_results = avg_results[:,4:6]
std_results = std_results[:,4:6]
avg_class_results = avg_class_results[4:6]
std_class_results = std_class_results[4:6]

In [ ]:
pr_header = [np.array(['vae', 'vae',             
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['pr_auc_mean','pr_auc_std',
          'pr_auc_mean','pr_auc_std',
          ])] 

In [ ]:
auc_header = [np.array([
                    'vae', 'vae',
                    'ed-vae', 'ed-vae',
                    ]), 
np.array(['auc_mean','auc_std',
          'auc_mean','auc_std',
          ])] 

### $ELBO$ 
Compare ELBO of VAE and ED-VAE


In [ ]:
# AUC
avg_auc_elbo_class_results = pd.DataFrame((avg_class_results[0,3], std_class_results[0,3], 
                                  avg_class_results[1,3], std_class_results[1,3],
                                  ), 
                                  index = auc_header).T
avg_auc_elbo_class_results = avg_auc_elbo_class_results.rename(index={0: "mean"})
avg_auc_elbo_final_results = pd.DataFrame((avg_results[:,0,3], std_results[:,0,3],
                                  avg_results[:,1,3], std_results[:,1,3], 
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_elbo_final_results.append(avg_auc_elbo_class_results)

vae              ed-vae          
      auc_mean   auc_std  auc_mean   auc_std
0     0.906740  0.007390  0.911173  0.007674
1     0.985576  0.002004  0.987114  0.001906
2     0.884042  0.007990  0.891146  0.007604
3     0.924055  0.009302  0.927871  0.008745
4     0.899538  0.013268  0.908718  0.013040
5     0.893064  0.008791  0.895072  0.007228
6     0.823253  0.012089  0.830588  0.011563
7     0.982244  0.003357  0.985006  0.002306
8     0.835084  0.011485  0.848064  0.012047
9     0.963205  0.007989  0.970858  0.006165
mean  0.909680  0.053844  0.915561  0.051721

In [ ]:
# PR-AUC
avg_pr_elbo_class_results = pd.DataFrame((avg_class_results[0,2], std_class_results[0,2], 
                                  avg_class_results[1,2], std_class_results[1,2],
                                  ), 
                                  index = pr_header).T
avg_pr_elbo_class_results = avg_pr_elbo_class_results.rename(index={0: "mean"})
avg_pr_elbo_final_results = pd.DataFrame((avg_results[:,0,2], std_results[:,0,2],
                                  avg_results[:,1,2], std_results[:,1,2],
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_elbo_final_results.append(avg_pr_elbo_class_results)

vae                 ed-vae           
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std
0       0.722367   0.016622    0.721989   0.018962
1       0.944248   0.007205    0.947529   0.007014
2       0.741947   0.010923    0.749466   0.012310
3       0.824475   0.018503    0.828265   0.018536
4       0.786449   0.018267    0.799682   0.017247
5       0.816291   0.007996    0.811389   0.007388
6       0.639569   0.023570    0.645248   0.022122
7       0.958223   0.006442    0.962574   0.004556
8       0.592811   0.020648    0.608472   0.021543
9       0.910692   0.014042    0.919511   0.011703
mean    0.793707   0.117990    0.799412   0.116207

### $R_{error}$ 
Compare Reconstruction Error of VAE and ED-VAE

In [ ]:
# AUC
avg_auc_rec_class_results = pd.DataFrame((avg_class_results[0,1], std_class_results[0,1], 
                                  avg_class_results[1,1], std_class_results[1,1],
                                  ), 
                                  index = auc_header).T
avg_auc_rec_class_results = avg_auc_rec_class_results.rename(index={0: "mean"})
avg_auc_rec_final_results = pd.DataFrame((avg_results[:,0,1], std_results[:,0,1], 
                                  avg_results[:,1,1], std_results[:,1,1],
                                  ), 
                                  index = auc_header, columns = index_labels).T
avg_auc_rec_final_results.append(avg_auc_rec_class_results)

vae              ed-vae          
      auc_mean   auc_std  auc_mean   auc_std
0     0.896063  0.007094  0.901122  0.007483
1     0.982152  0.002555  0.985460  0.002292
2     0.878164  0.008535  0.886226  0.007555
3     0.915553  0.007754  0.920695  0.007345
4     0.901077  0.013091  0.911860  0.012613
5     0.884665  0.008748  0.885300  0.007125
6     0.819804  0.014603  0.826755  0.013618
7     0.977047  0.004266  0.981493  0.002973
8     0.812620  0.010624  0.827066  0.011497
9     0.959224  0.010769  0.969056  0.007599
mean  0.902637  0.056620  0.909503  0.054988

In [ ]:
# PR-AUC
avg_pr_rec_class_results = pd.DataFrame((avg_class_results[0,0], std_class_results[0,0], 
                                  avg_class_results[1,0], std_class_results[1,0],
                                  ), 
                                  index = pr_header).T
avg_pr_rec_class_results = avg_pr_rec_class_results.rename(index={0: "mean"})
avg_pr_rec_final_results = pd.DataFrame((avg_results[:,0,0], std_results[:,0,0], 
                                  avg_results[:,1,0], std_results[:,1,0],
                                  ), 
                                  index = pr_header, columns = index_labels).T
avg_pr_rec_final_results.append(avg_pr_rec_class_results)

vae                 ed-vae           
     pr_auc_mean pr_auc_std pr_auc_mean pr_auc_std
0       0.686710   0.017583    0.683710   0.019027
1       0.936850   0.007662    0.943020   0.007366
2       0.721591   0.013055    0.727865   0.014579
3       0.801492   0.017408    0.805554   0.017027
4       0.780542   0.019491    0.793460   0.018910
5       0.809740   0.008844    0.802923   0.007537
6       0.631358   0.022082    0.635030   0.021455
7       0.950122   0.008087    0.956836   0.005809
8       0.549982   0.020126    0.564549   0.021504
9       0.904508   0.016123    0.914632   0.013445
mean    0.777290   0.126473    0.782758   0.126155

## Formated for Publication
Mean and SEM

#### $ELBO$ 
Compare ELBO of VAE and ED-VAE

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_elbo_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_elbo_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,90.7±0.2,91.1±0.2
1,98.6±0.1,98.7±0.1
2,88.4±0.3,89.1±0.2
3,92.4±0.3,92.8±0.3
4,90.0±0.4,90.9±0.4
5,89.3±0.3,89.5±0.2
6,82.3±0.4,83.1±0.4
7,98.2±0.1,98.5±0.1
8,83.5±0.4,84.8±0.4
9,96.3±0.3,97.1±0.2


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,90.7±0.2,91.1±0.2
1,98.6±0.1,98.7±0.1
2,88.4±0.3,89.1±0.2
3,92.4±0.3,92.8±0.3
4,90.0±0.4,90.9±0.4
5,89.3±0.3,89.5±0.2
6,82.3±0.4,83.1±0.4
7,98.2±0.1,98.5±0.1
8,83.5±0.4,84.8±0.4
9,96.3±0.3,97.1±0.2


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_elbo_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_elbo_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_final_results.keys().get_level_values(0)[::2]).T  
df = df.append(pd.DataFrame((
    #round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_elbo_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_elbo_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_elbo_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_elbo_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,72.2±0.5,72.2±0.6
1,94.4±0.2,94.8±0.2
2,74.2±0.3,74.9±0.4
3,82.4±0.6,82.8±0.6
4,78.6±0.6,80.0±0.5
5,81.6±0.3,81.1±0.2
6,64.0±0.7,64.5±0.7
7,95.8±0.2,96.3±0.1
8,59.3±0.7,60.8±0.7
9,91.1±0.4,92.0±0.4


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,72.2±0.5,72.2±0.6
1,94.4±0.2,94.8±0.2
2,74.2±0.3,74.9±0.4
3,82.4±0.6,82.8±0.6
4,78.6±0.6,80.0±0.5
5,81.6±0.3,81.1±0.2
6,64.0±0.7,64.5±0.7
7,95.8±0.2,96.3±0.1
8,59.3±0.7,60.8±0.7
9,91.1±0.4,92.0±0.4


### $R_{error}$
Compare Reconstruction Error of VAE and ED-VAE

In [ ]:
# AUC
df = pd.DataFrame((
    round(avg_auc_rec_final_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_final_results[column,'auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_auc_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) + '±' + round(avg_auc_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_auc_rec_class_results[column,'auc_mean']*100,1).astype(str) for column in avg_auc_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_auc_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,89.6±0.2,90.1±0.2
1,98.2±0.1,98.5±0.1
2,87.8±0.3,88.6±0.2
3,91.6±0.2,92.1±0.2
4,90.1±0.4,91.2±0.4
5,88.5±0.3,88.5±0.2
6,82.0±0.5,82.7±0.4
7,97.7±0.1,98.1±0.1
8,81.3±0.3,82.7±0.4
9,95.9±0.3,96.9±0.2


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,89.6±0.2,90.1±0.2
1,98.2±0.1,98.5±0.1
2,87.8±0.3,88.6±0.2
3,91.6±0.2,92.1±0.2
4,90.1±0.4,91.2±0.4
5,88.5±0.3,88.5±0.2
6,82.0±0.5,82.7±0.4
7,97.7±0.1,98.1±0.1
8,81.3±0.3,82.7±0.4
9,95.9±0.3,96.9±0.2


In [ ]:
# PR-AUC
df = pd.DataFrame((
    round(avg_pr_rec_final_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_final_results[column,'pr_auc_std']*100/np.sqrt(reps),1).astype(str) for column in avg_pr_rec_final_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_final_results.keys().get_level_values(0)[::2]).T
df = df.append(pd.DataFrame((
    #round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) + '±' + round(avg_pr_rec_class_results[column,'pr_auc_std']*100/np.sqrt(reps * len(df.index)),1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    round(avg_pr_rec_class_results[column,'pr_auc_mean']*100,1).astype(str) for column in avg_pr_rec_class_results.keys().get_level_values(0)[::2]
    ),index = avg_pr_rec_class_results.keys().get_level_values(0)[::2]).T)
df

,vae,ed-vae
0,68.7±0.6,68.4±0.6
1,93.7±0.2,94.3±0.2
2,72.2±0.4,72.8±0.5
3,80.1±0.6,80.6±0.5
4,78.1±0.6,79.3±0.6
5,81.0±0.3,80.3±0.2
6,63.1±0.7,63.5±0.7
7,95.0±0.3,95.7±0.2
8,55.0±0.6,56.5±0.7
9,90.5±0.5,91.5±0.4


In [ ]:
df.style.apply(highlight_max, axis=1)

,vae,ed-vae
0,68.7±0.6,68.4±0.6
1,93.7±0.2,94.3±0.2
2,72.2±0.4,72.8±0.5
3,80.1±0.6,80.6±0.5
4,78.1±0.6,79.3±0.6
5,81.0±0.3,80.3±0.2
6,63.1±0.7,63.5±0.7
7,95.0±0.3,95.7±0.2
8,55.0±0.6,56.5±0.7
9,90.5±0.5,91.5±0.4
